In [10]:
import cv2
import dlib
import winsound
from scipy.spatial import distance

In [11]:
PATH_SOUND = "assets\\beep-01a"
PATH_FACELANDMARK = "assets\\shape_predictor_68_face_landmarks.dat"
CATCH_INTERVAL_ITER = 10
iteration = 0

In [12]:
def beep():
    winsound.PlaySound(PATH_SOUND, winsound.SND_FILENAME | winsound.SND_ASYNC)
    
def warn():
    cv2.putText(frame, "Buka mata anda", (50, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 100), 3)
    cv2.putText(frame, "Rawan Mengantuk", (50, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 3)
    if iteration == 0 : beep()
        

def show_verbose(ct, eye_ratio):
    cv2.putText(frame, str(ct), (50, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (51, 56, 255), 3)
    cv2.putText(frame, str(eye_ratio), (100, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (21, 56, 255), 3)

In [13]:
def get_eye_ratio(eye):
    x = distance.euclidean(eye[2], eye[4]) # P3 P5
    y = distance.euclidean(eye[1], eye[5]) # P2 P6
    z = distance.euclidean(eye[0], eye[3]) # P1 P4
    aspect_ratio_eye = (x + y) / (2 * z)
    return aspect_ratio_eye

def left_eye_to_frame(face_landmarks):
    left_eye = []
    for i in range(36, 42):
        x = face_landmarks.part(i).x
        y = face_landmarks.part(i).y
        left_eye.append((x, y))
        next_i = i + 1
        if i == 41: next_i = 36 
        next_x = face_landmarks.part(next_i).x
        next_y = face_landmarks.part(next_i).y
        cv2.line(frame, (x, y), (next_x, next_y), (0, 255, 255), 1)
    return left_eye

def right_eye_to_frame(face_landmarks):
    right_eye = []
    for i in range(42, 48):
        x = face_landmarks.part(i).x
        y = face_landmarks.part(i).y
        right_eye.append((x, y))
        next_i = i + 1
        if i == 47: next_i = 42 
        next_x = face_landmarks.part(next_i).x
        next_y = face_landmarks.part(next_i).y
        cv2.line(frame, (x, y), (next_x, next_y), (0, 255, 255), 1)
    return right_eye
    

In [14]:
cap = cv2.VideoCapture(0)

face_detector = dlib.get_frontal_face_detector()

face_predictor = dlib.shape_predictor(PATH_FACELANDMARK)

ct = 0
while True:
    iteration += 1
    ret, frame = cap.read()
    gsc = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_detector(gsc)
    if not faces: 
        cv2.putText(frame, "Wajah tidak terdeteksi", (50, 400), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 100), 3)
    for face in faces:
        face_landmarks = face_predictor(gsc, face)
        right_eye = right_eye_to_frame(face_landmarks)
        left_eye = left_eye_to_frame(face_landmarks)
        
    right_eye_ratio = get_eye_ratio(right_eye)
    left_eye_ratio = get_eye_ratio(left_eye)
    eye_ratio = round((left_eye_ratio + right_eye_ratio) / 2, 2)
    show_verbose(ct, eye_ratio)
    
    if iteration == CATCH_INTERVAL_ITER:
        iteration = 0
        if eye_ratio > 0.25: ct -= 1
        else: ct += 1
        ct = max(ct, 0)
    if ct > 5:
        warn()
        
    cv2.imshow("prototype", frame)
    key = cv2.waitKey(1)
    if key == ord('e'): break
        
cap.release()
cv2.destroyAllWindows()
